In [2]:
 # todo clean up this import list
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import datetime
from geopy.geocoders import Nominatim # city lat long
from geopy.point import Point

In [3]:
# load the cities data, remove small cities, sort by population, and re-index


data_path = 'data/City_Population_in_Iowa_by_County_and_Year.csv'
city_df = pd.read_csv(data_path)

print(city_df.head())

N_cities = 30
population_cutoff = 5000

drop_row_idxs = []  # remove small cities
for idx,row in city_df.iterrows():
    population = row['Estimate']
    if population < population_cutoff:
        city_df.drop(idx,inplace=True)
    elif pd.isna(population):
        city_df.drop(idx,inplace=True) # drop nans
    elif 'Balance of' in row['City']:
        city_df.drop(idx,inplace=True) # drop non-city metro area entries
    # update time formatting
        
# sort by population
city_df = city_df.sort_values('Estimate',ascending=False)

# get indices for the trimmed, re-ordered dataframe
city_df.reset_index(drop=True,inplace=True)



      FIPS Primary County Coordinates County         City           Year  \
0  1900370  (41.3307464, -94.4709413)  Adair  Adair (pt.)  April 01 2010   
1  1908425  (41.3307464, -94.4709413)  Adair  Bridgewater  April 01 2010   
2  1911395  (41.3307464, -94.4709413)  Adair  Casey (pt.)  April 01 2010   
3  1928290  (41.3307464, -94.4709413)  Adair   Fontanelle  April 01 2010   
4  1932790  (41.3307464, -94.4709413)  Adair   Greenfield  April 01 2010   

   Estimate  
0     762.0  
1     182.0  
2      10.0  
3     672.0  
4    1982.0  


In [4]:
# clean up duplicate rows based on data from multiple years (and re-index)
# warning - dataframe in-place operations


# add a column with datetime conversion from year
datetime_column = []
for idx,row in city_df.iterrows():
    freshness_string = row['Year']
    freshness_datetime = datetime.strptime(freshness_string, '%B %d %Y').isoformat()
    datetime_column.append(freshness_datetime)
city_df['DataFreshness'] = datetime_column
# todo drop un-used columns

unique_cities = {}
drop_list = []
# remove duplicates
for idx,row in city_df.iterrows():
    city_name = row['City']
    data_freshness = row['DataFreshness']
    #print(city_name, data_freshness)
    
    if not(city_name in unique_cities.keys()):  # first time we've seen data for this city?
        unique_cities[city_name] = (data_freshness, idx)    
        
    elif (city_name in unique_cities.keys()):
        if data_freshness > unique_cities[city_name][0]: # if this data is more recent, replace it
            
            stale_idx = unique_cities[city_name][1]
            city_df.drop(stale_idx, inplace=True) # drop the old stale city data
            #drop_list.append(stale_idx)
            
            unique_cities[city_name] = (data_freshness, idx) # update helper dict with the fresh data
            
        else: # this city is already visited and stale
            city_df.drop(idx, inplace=True)
            #drop_list.append(idx)

#drop_list = np.sort(drop_list)
#for idx in drop_list:
#    city_df.drop(idx,inplace=True)

#print(unique_cities)

# get indices for the trimmed, re-ordered dataframe
city_df.reset_index(drop=True,inplace=True)

print(city_df.head())
print(city_df['City'])


      FIPS Primary County Coordinates    County              City  \
0  1921000  (41.6855048, -93.5735335)      Polk  Des Moines (pt.)   
1  1912000  (42.0789478, -91.5989646)      Linn      Cedar Rapids   
2  1919000  (41.6370978, -90.6232428)     Scott         Davenport   
3  1973335  (42.3897169, -96.0447673)  Woodbury  Sioux City (pt.)   
4  1938595  (41.6715511, -91.5880849)   Johnson         Iowa City   

           Year  Estimate        DataFreshness  
0  July 01 2017  217295.0  2017-07-01T00:00:00  
1  July 01 2017  132228.0  2017-07-01T00:00:00  
2  July 01 2017  102320.0  2017-07-01T00:00:00  
3  July 01 2017   82508.0  2017-07-01T00:00:00  
4  July 01 2017   75798.0  2017-07-01T00:00:00  
0          Des Moines (pt.)
1              Cedar Rapids
2                 Davenport
3          Sioux City (pt.)
4                 Iowa City
5                  Waterloo
6                      Ames
7                    Ankeny
8            Council Bluffs
9                   Dubuque
10    West 

In [5]:
# todo - remove '(pt.)' suffixes

names = []
for idx,row in city_df.iterrows():
    name = row['City']
    if '(pt.)' in name:
        name = name.split('(pt.)')[0]
        name = name[:-1]
    names.append(name)
    
city_df['City'] = names

city_df.head()





,FIPS,Primary County Coordinates,County,City,Year,Estimate,DataFreshness
0,1921000,"(41.6855048, -93.5735335)",Polk,Des Moines,July 01 2017,217295.0,2017-07-01T00:00:00
1,1912000,"(42.0789478, -91.5989646)",Linn,Cedar Rapids,July 01 2017,132228.0,2017-07-01T00:00:00
2,1919000,"(41.6370978, -90.6232428)",Scott,Davenport,July 01 2017,102320.0,2017-07-01T00:00:00
3,1973335,"(42.3897169, -96.0447673)",Woodbury,Sioux City,July 01 2017,82508.0,2017-07-01T00:00:00
4,1938595,"(41.6715511, -91.5880849)",Johnson,Iowa City,July 01 2017,75798.0,2017-07-01T00:00:00


In [9]:
# get lat/long for cities (not just their county)
#view_box_topleft = (43.5473, 96.7283) # 43.5473° N, 96.7283° W # sioux falls
#view_box_bottomright = (39.7817, 89.6501)  # Springfield IL : 39.7817° N, 89.6501° W
geolocator = Nominatim(user_agent="iowa_cities")
                    #   view_box = (Point(view_box_topleft), Point(view_box_bottomright)),
                    #   bounded=True)


# todo
latitudes = []
longitudes = []
for idx,row in city_df.iterrows():
    name = row['City'] + ' Iowa USA'
    print(name)
    location = geolocator.geocode(name)
    print(location)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude) # WARNING sign conventions
    
city_df['Latitude'] = latitudes
city_df['Longitude'] = longitudes

city_df.head()
    

Des Moines Iowa USA
Des Moines, Polk County, Iowa, USA
Cedar Rapids Iowa USA
Cedar Rapids, Iowa, USA
Davenport Iowa USA
Davenport, Scott County, Iowa, USA
Sioux City Iowa USA
Sioux City, Woodbury County, Iowa, USA
Iowa City Iowa USA
Iowa City, Johnson County, Iowa, USA
Waterloo Iowa USA
Waterloo, Iowa, USA
Ames Iowa USA
Ames, Story County, Iowa, USA
Ankeny Iowa USA
Ankeny, Polk County, Iowa, USA
Council Bluffs Iowa USA
Council Bluffs, Pottawattamie County, Iowa, USA
Dubuque Iowa USA
Dubuque, Dubuque County, Iowa, USA
West Des Moines Iowa USA
West Des Moines, Polk County, Iowa, USA
Cedar Falls Iowa USA
Cedar Falls, Iowa, 50613, USA
Marion Iowa USA
Marion, Linn County, Iowa, 52302, USA
Bettendorf Iowa USA
Bettendorf, Scott County, Iowa, 52722, USA
Urbandale Iowa USA
Urbandale, Polk County, Iowa, USA
Mason City Iowa USA
Mason City, Cerro Gordo County, Iowa, USA
Marshalltown Iowa USA
Marshalltown, Marshall County, Iowa, 50158, USA
Clinton Iowa USA
Clinton, Clinton County, Iowa, USA
Burling

,FIPS,County,City,Estimate,DataFreshness,Latitude,Longitude
0,1921000,Polk,Des Moines,217295.0,2017-07-01T00:00:00,41.591064,-93.603715
1,1912000,Linn,Cedar Rapids,132228.0,2017-07-01T00:00:00,41.975887,-91.670405
2,1919000,Scott,Davenport,102320.0,2017-07-01T00:00:00,41.523644,-90.577637
3,1973335,Woodbury,Sioux City,82508.0,2017-07-01T00:00:00,42.496682,-96.405878
4,1938595,Johnson,Iowa City,75798.0,2017-07-01T00:00:00,41.661256,-91.529911


In [10]:

city_df.drop(labels='Primary County Coordinates', axis=1, inplace=True)
city_df.drop(labels='Year', axis=1, inplace=True)

city_df.head()

ValueError: labels ['Primary County Coordinates'] not contained in axis

In [11]:
# write 
city_file_target = 'data/iowa_cities_clean.json'
city_df.to_json(orient='records',path_or_buf = city_file_target)